In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from collections import Counter
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_recall_curve, PrecisionRecallDisplay
from sklearn import metrics
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler


In [14]:
df = pd.read_csv("Labelled_data/uwv_data_with_labels_pattern_1.csv", encoding_errors="ignore", on_bad_lines='skip', sep=",")
df["TIMESTAMP"] = pd.to_datetime(df["TIMESTAMP"], infer_datetime_format=True)

C:\Users\jveldman\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (12,14,15,16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [15]:
#df_raw = df.sort_values(by=["Label", "SessionID", "TIMESTAMP"], ascending=[False, True, True])
df_raw = df.sort_values(by=["SessionID", "TIMESTAMP"])
df_raw

,Unnamed: 0,CustomerID,AgeCategory,Gender,Office_U,Office_W,SessionID,IPID,TIMESTAMP,VHOST,URL_FILE,PAGE_NAME,REF_URL_category,page_load_error,page_action_detail,tip,service_detail,xps_info,Label
3273278,3273278,1837760,50-65,M,247,247,46,1419591,2015-11-06 08:07:22.780,digid.werk.nl,/werk_nl/werknemer/mijn_werkmap/doorgeven/taken,taken,NaN,0,NaN,NaN,NaN,NaN,normal
5333642,5333642,1837760,50-65,M,247,247,46,1419591,2015-11-06 08:07:40.767,digid.werk.nl,/werk_nl/werknemer/mijn_werkmap/doorgeven/mijn...,mijn_sollicitaties,NaN,0,NaN,NaN,NaN,NaN,normal
3733243,3733243,1837760,50-65,M,247,247,46,1419591,2015-11-06 08:07:51.390,digid.werk.nl,/werk_nl/werknemer/mijn_werkmap/postvak/mijn_d...,mijn_documenten,NaN,0,NaN,NaN,NaN,NaN,normal
5904405,5904405,1837760,50-65,M,247,247,46,1419591,2015-11-06 08:08:06.003,digid.werk.nl,/werk_nl/werknemer/mijn_werkmap/postvak/mijn_b...,mijn_berichten,NaN,0,NaN,NaN,NaN,NaN,normal
5573282,5573282,1837760,50-65,M,247,247,46,1419591,2015-11-06 08:08:19.343,digid.werk.nl,/werk_nl/werknemer/mijn_werkmap/postvak/mijn_b...,mijn_berichten,NaN,0,NaN,NaN,NaN,NaN,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1613546,1613546,2209110,18-29,V,230,230,55314751,505057,2016-02-28 08:17:15.947,www.werk.nl,/werk_nl/werknemer/werkmap,werkmap,UWV,0,NaN,NaN,NaN,NaN,normal
2925569,2925569,2209110,18-29,V,230,230,55314751,505057,2016-02-28 08:18:10.877,digid.werk.nl,/werk_nl/werknemer/mijn_werkmap/doorgeven/taken,taken,NaN,0,NaN,NaN,NaN,NaN,normal
4068602,4068602,2209110,18-29,V,230,230,55314751,505057,2016-02-28 08:18:41.493,digid.werk.nl,/werk_nl/werknemer/mijn_werkmap/doorgeven/taken,taken,NaN,0,NaN,NaN,NaN,NaN,normal
5013440,5013440,2209110,18-29,V,230,230,55314751,505057,2016-02-28 08:19:59.737,digid.werk.nl,/werk_nl/werknemer/mijn_werkmap/doorgeven/taken,taken,NaN,0,NaN,NaN,NaN,NaN,normal


In [16]:
def concat_and_filter(df_raw, file_name, amount_real, amount_anomalies):

    df_gen_anomalies = pd.read_csv(file_name)
    df_gen_anomalies["Label"] = "potential_anomaly"
    df_raw_filtered = df_raw[df_raw["SessionID"].isin(df_raw["SessionID"].unique()[:amount_real])]
    df_gen_anomalies_filtered = df_gen_anomalies[df_gen_anomalies["SessionID"].isin(df_gen_anomalies["SessionID"].unique()[:amount_anomalies])]

    df = pd.concat([df_raw_filtered, df_gen_anomalies_filtered])
    return df

def df_one_hot_encoder(df):
    #creating instance of one-hot-encoder
    encoder = OneHotEncoder()
    
    #perform one-hot encoding on 'team' column 
    encoder_df = pd.DataFrame(encoder.fit_transform(df[['URL_FILE']]).toarray(), index=df.index)
    encoder_df.columns = encoder.get_feature_names(['URL_FILE'])

    #merge one-hot encoded columns back with original DataFrame
    final_df = df.join(encoder_df)
    return final_df

def calculating_duration(df):
    #Calculating the duration of an activity by using the end time. Duration returned in minutes
    df["End_time"] = df.groupby("SessionID")["TIMESTAMP"].shift(periods=-1)
    df["Duration"] = (df["End_time"] - df["TIMESTAMP"]) / pd.offsets.Minute(1)
    return df

def make_df_for_isolation_forest(df):
    #Counting each activity in every session
    df_sum = df.groupby("SessionID").sum()
    df_sum["Label"] = df.groupby("SessionID")["Label"].max()
    return df_sum

In [17]:
#Filter to get df with only real anomalies
amount_for_filtering_real_anomalies = 0
df_only_real_anomalies = df_raw[df_raw["Label"] == "potential_anomaly"]

#Make the data ready for training
df_only_anomaly_selection = df_only_real_anomalies[["SessionID", "URL_FILE", "Label"]]
one_hot_encoded_df_only_anomalies = df_one_hot_encoder(df_only_anomaly_selection)
df_only_anomalies_for_training = make_df_for_isolation_forest(one_hot_encoded_df_only_anomalies)

df_only_anomalies_for_train_set = df_only_anomalies_for_training[:10]
df_only_anomalies_for_test_set = df_only_anomalies_for_training[10:]

In [18]:
#Select amount of sessions used in the dataframe
var_1_amount_real = 50000
var_1_amount_generated = 0
df_real = concat_and_filter(df_raw, "generated_data/Pattern_2/generated_anomalies_plus_0_5_2k.csv", amount_real = var_1_amount_real, amount_anomalies = var_1_amount_generated)

#Filter only the normal instances. 
df_real = df_real[df_real["Label"] == "normal"]

#df_real = pd.concat([df_real, df_only_real_anomalies_filtered])

#Make the data ready for training
df_real_selection = df_real[["SessionID", "URL_FILE", "Label"]]
one_hot_encoded_df_real = df_one_hot_encoder(df_real_selection)
df_real_for_training = make_df_for_isolation_forest(one_hot_encoded_df_real)

In [19]:
#Make train test split
X = df_real_for_training.drop(columns=["Label"])
y = df_real_for_training["Label"]

X_train_temp, X_test_temp, y_train_temp, y_test_temp = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
#Add 10 anomalies back to the training data to make it more realistic.
X_train = pd.concat([X_train_temp, df_only_anomalies_for_train_set.drop(columns=["Label"])]).fillna(0)
y_train = pd.concat([y_train_temp, df_only_anomalies_for_train_set["Label"]])

In [21]:
X_test = pd.concat([X_test_temp, df_only_anomalies_for_test_set.drop(columns=["Label"])]).fillna(0)
y_test = pd.concat([y_test_temp, df_only_anomalies_for_test_set["Label"]])

In [26]:
print(sorted(Counter(y_train).items()))

[('normal', 39957), ('potential_anomaly', 10)]


In [22]:
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

print(sorted(Counter(y_resampled).items()))

[('normal', 39957), ('potential_anomaly', 39957)]


In [23]:
clf_oversample_test = LogisticRegression(max_iter=1000).fit(X_resampled, y_resampled)

In [24]:
y_test_predict = clf_oversample_test.predict(X_test)

print(classification_report(y_test, y_test_predict))

                   precision    recall  f1-score   support

           normal       0.99      1.00      0.99      9990
potential_anomaly       0.99      0.78      0.87       450

         accuracy                           0.99     10440
        macro avg       0.99      0.89      0.93     10440
     weighted avg       0.99      0.99      0.99     10440



In [25]:
#Add the generated anomalies to the training dataset
X_train_extra = X_resampled
y_train_extra = y_resampled

#transform y values to binary class
Le = LabelEncoder()
Le.fit(y_train_extra)
y_train_transformed = Le.transform(y_train)
y_test_transformed = Le.transform(y_test)
y_train_extra_transformed = Le.transform(y_train_extra)

#Make sure that both dataframes have the same columns
for column_name in X_train_extra.columns:
    if column_name not in X_test.columns:
        X_test[column_name] = 0

#Make sure that both dataframes have the same columns
for column_name in X_test.columns:
    if column_name not in X_train_extra.columns:
        X_test.drop(columns=[column_name], inplace=True)

#clf_extra = tree.DecisionTreeClassifier().fit(X_train_extra, y_train_extra_transformed)
clf_extra = LogisticRegression(random_state=0, max_iter=1000).fit(X_train_extra, y_train_extra_transformed)
#y_train_predict = clf_extra.predict(X_train_extra)
y_test_predict = clf_extra.predict(X_test)

#AUC predict
fpr, tpr, thresholds = metrics.roc_curve(y_test_transformed, y_test_predict)
print(metrics.auc(fpr, tpr))

0.8876276276276276
